### Deep Learning Barrier Option

We used Numba and CuPy in the previous notebook to run Monte Carlo simulation to determine the price of the Asian Barrier option. A Monte Carlo simulation needs millions of paths to get an accurate answer which is computationally intensive. [Ryan et al (2018)](https://arxiv.org/abs/1809.02233) showed that a deep learning model can be trained to value derivatives. The deep learning model is accurate and very fast, capable of producing valuations a million times faster than traditional models. In the this notebook, we will use a fully connected network to learn the pricing mode of the Asian Barrier option. Monte Carlo simulation is used as pricing ground truth for the training. We use the same Asian Barrier Option model as last notebook with parameters listed as following:

```
T - Maturity (yrs.)
S - Spot (usd)
K - Strike (usd)
sigma - Volatility (per.)
r - Risk Free Rate (per.)
mu - Stock Drift Rate (per.)
B - Barrier (usd)
```

### Batched Data generation

The dataset is an important part of the Deep learning training. We will modify the previous single Asian Barrier Option pricing code to handle a batch of Barrier Option pricing. 

Loading all the necessary libraries:-

In [1]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  10063      0 --:--:-- --:--:-- --:--:-- 10063
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9MB 82kB/s 
     |████████████████████████████████| 1.0MB 36.7MB/s 
+ set +ex
Installation succeeded!


In [ ]:
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack


In [ ]:
cupy_barrier_option = cupy.RawKernel(r'''
extern "C" __global__ void barrier_option(
    float *d_s,
    const float T,
    const float K,
    const float B,
    const float S0,
    const float sigma,
    const float mu,
    const float r,
    const float * d_normals,
    const long N_STEPS,
    const long N_PATHS)
{
  unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
  unsigned stride = blockDim.x * gridDim.x;
  unsigned tid = threadIdx.x;
 
  const float tmp1 = mu*T/N_STEPS;
  const float tmp2 = exp(-r*T);
  const float tmp3 = sqrt(T/N_STEPS);
  double running_average = 0.0;
 
  for (unsigned i = idx; i<N_PATHS; i+=stride)
  {
    float s_curr = S0;
    unsigned n=0;
    for(unsigned n = 0; n < N_STEPS; n++){
       s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS];
       running_average += (s_curr - running_average) / (n + 1.0) ;
       if (running_average <= B){
           break;
       }
    }
 
    float payoff = (running_average>K ? running_average-K : 0.f);
    d_s[i] = tmp2 * payoff;
  }
}
 
''', 'barrier_option')

In [ ]:
def get_option_price(T, K, B, S0, sigma, mu, r, N_PATHS = 8192000, N_STEPS = 365, seed=3):
    number_of_threads = 256
    number_of_blocks = (N_PATHS-1) // number_of_threads + 1
    cupy.random.seed(seed)
    randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
    output =  cupy.zeros(N_PATHS, dtype=cupy.float32)
    cupy_barrier_option((number_of_blocks,), (number_of_threads,),
                   (output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r),  randoms_gpu, N_STEPS, N_PATHS))
    v = output.mean()
    out_df = cudf.DataFrame()
    out_df['p'] = cudf.Series([v.item()])
    return out_df 

In [ ]:
import pandas as cudf
get_option_price(1, 100, 95, 100, 0.2, 0, 0, N_PATHS = 819200, N_STEPS = 365, seed=3)

,p
0,4.411485


In [ ]:
import numba
from numba import cuda

@cuda.jit
def batch_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_BATCH):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp3 = math.sqrt(T/N_STEPS)
    for i in range(ii, N_PATHS * N_BATCH, stride):
        batch_id = i // N_PATHS
        path_id = i % N_PATHS
        tmp1 = mu[batch_id]*T/N_STEPS
        tmp2 = math.exp(-r[batch_id]*T)
        running_average = 0.0
        s_curr = S0[batch_id]
        for n in range(N_STEPS):

            s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            if i==0 and batch_id == 2:
                print(s_curr)
            if running_average <= B[batch_id]:
                break
        payoff = running_average - K[batch_id] if running_average > K[batch_id] else 0
        d_s[i] = tmp2 * payoff

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, threads=512, seed=15):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.T = np.float32(1.0)
        self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration
        #X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
        # scale the [0, 1) random numbers to the correct range for each of the option parameters
        X = cupy.array([100.0, 95, 100, 0.2, 0, 0], dtype=cupy.float32)
        # make sure the Barrier is smaller than the Strike price
        #X[1] = X[0] * X[1]
        randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        batch_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, X[0], 
                              X[1], X[2], X[3], X[4], X[5], randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH)
        o = self.output.reshape(self.N_BATCH, self.N_PATHS)
        Y = o.mean(axis = 1) 
        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))
ds = NumbaOptionDataSet(10, number_path=100000, batch=16, seed=15)
for i in ds:
    print(i[1])

tensor([4.3987, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')
tensor([4.4298, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')
tensor([4.3900, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')
tensor([4.4235, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')
tensor([4.4219, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')
tensor([4.3761, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       dev

We can implement the same code by using Numba to accelerate the calculation in GPU:-

In [ ]:
#%%writefile cupy_dataset.py
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

@cuda.jit
def single_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_STOCKS, s_curr):

    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)    

    for i in range(ii, N_PATHS, stride): # for each path          
        running_average = 0.0

        for j in range(N_STOCKS): # initialize S0
            s_curr[j] = S0[j]

        for n in range(N_STEPS): # for each step
            s_curr_avg = 0.0

            for j in range(N_STOCKS): # for each stock
                tmp1 = mu[j]*T/N_STEPS  
                s_curr[j] += tmp1 * s_curr[j] + sigma[j]*s_curr[j]*tmp3*d_normals[i,n,j]
                s_curr_avg = s_curr_avg + 1.0/(j + 1.0) * (s_curr[j] - s_curr_avg) # S average in this step

            # add stock average to running average
            running_average = running_average + 1.0/(n + 1.0) * (s_curr_avg - running_average)

            # compare to barrier
            if running_average <= B:
                break

        payoff = running_average - K if running_average > K else 0
        d_s[i] = tmp2 * payoff
    

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, number_stocks = 3, batch=1, threads=512, seed=15, T=1):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_STOCKS = number_stocks
        self.N_BATCH = batch
        self.T = np.float32(T)
        self.output = cupy.zeros(self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_STOCKS - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)

        ############ <new
        self.Z_mean = cupy.zeros(self.N_STOCKS, dtype=cupy.float32)
        self.Z_cov = (-0.2 + cupy.random.rand(self.N_STOCKS*self.N_STOCKS, dtype=cupy.float32)*0.4).reshape(self.N_STOCKS,self.N_STOCKS)
        cupy.fill_diagonal(self.Z_cov, 1)
        ############ new>

    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration

        X = cupy.zeros((self.N_BATCH, 3 + self.N_STOCKS * 3), dtype=cupy.float32)
        Y = cupy.zeros(self.N_BATCH, dtype=cupy.float32)

        for i in range(self.N_BATCH): # for each batch
          self.S0 = cupy.random.rand(self.N_STOCKS, dtype=cupy.float32) * 200
          self.K = 110.0
          self.B = 100.0
          self.sigma = cupy.random.rand(self.N_STOCKS, dtype=cupy.float32) * 0.2
          self.mu = cupy.random.rand(self.N_STOCKS, dtype=cupy.float32) * 0.2
          self.r = 0.05
          self.s_curr = cupy.zeros(self.N_STOCKS, dtype=cupy.float32) # used to store s_curr in kernel

          ############ <new - add correlation between stocks
          all_normals = cupy.random.multivariate_normal(self.Z_mean, self.Z_cov, (self.N_PATHS, self.N_STEPS), dtype=cupy.float32)
          ############ new>
          
          single_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, self.K, self.B, self.S0, 
                                                                                    self.sigma, self.mu, self.r, all_normals, self.N_STEPS, self.N_PATHS, self.N_STOCKS, self.s_curr)
          Y[i] = self.output.mean()

          ############ <new - combine to get X matrix
          X[i,:] = cupy.array([self.K, self.B] + self.S0.tolist() +
                                self.sigma.tolist() + self.mu.tolist() + [self.r], dtype=cupy.float32)
          ############ new>
        
        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

ds = NumbaOptionDataSet(max_len=10, number_path=100, batch=16, seed=15)
for i in ds:
  print(i[1])

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       

In [4]:
#%%writefile cupy_dataset.py
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

@cuda.jit
def single_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_STOCKS, s_curr):

    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)

    for i in range(ii, N_PATHS, stride): # for each path          
        running_average = 0.0

        for j in range(N_STOCKS): # initialize S0
            s_curr[j] = S0[j]

        for n in range(N_STEPS): # for each step
            s_curr_avg = 0.0

            for j in range(N_STOCKS): # for each stock
                tmp1 = mu[j]*T/N_STEPS  
                s_curr[j] += tmp1 * s_curr[j] + sigma[j]*s_curr[j]*tmp3*d_normals[i,n,j]
                s_curr_avg = s_curr_avg + 1.0/(j + 1.0) * (s_curr[j] - s_curr_avg) # S average in this step

            # add stock average to running average
            running_average = running_average + 1.0/(n + 1.0) * (s_curr_avg - running_average)

            # compare to barrier
            if running_average <= B:
                break

        payoff = running_average - K if running_average > K else 0
        d_s[i] = tmp2 * payoff
    

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, number_stocks = 2, batch=1, threads=512, seed=15, T=1):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_STOCKS = number_stocks
        self.N_BATCH = batch
        self.T = np.float32(T)
        self.output = cupy.zeros(self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)

        ############ <new
        self.Z_mean = cupy.zeros(self.N_STOCKS, dtype=cupy.float32)
        #self.Z_cov = (-0.2 + cupy.random.rand(self.N_STOCKS*self.N_STOCKS, dtype=cupy.float32)*0.4).reshape(self.N_STOCKS,self.N_STOCKS)
        #cupy.fill_diagonal(self.Z_cov, 1)
        #self.Z_cov = cupy.ones([self.N_STOCKS,self.N_STOCKS], dtype=cupy.float32)
        self.Z_cov = cupy.array([[1,0.99999],[0.99999,1]], dtype=cupy.float32)
        ############ new>

    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration

        X = cupy.zeros((self.N_BATCH, 3 + self.N_STOCKS * 3), dtype=cupy.float32)
        Y = cupy.zeros(self.N_BATCH, dtype=cupy.float32)

        for i in range(self.N_BATCH): # for each batch
            self.S0 = cupy.array([100]*self.N_STOCKS, dtype=cupy.float32)
            self.K = 100.0
            self.B = 95.0
            self.sigma = cupy.array([0.2]*self.N_STOCKS, dtype=cupy.float32)
            self.mu = cupy.array([0]*self.N_STOCKS, dtype=cupy.float32)
            self.r = 0.05
            self.s_curr = cupy.zeros(self.N_STOCKS, dtype=cupy.float32) # used to store s_curr in kernel

            ############ <new - add correlation between stocks
            all_normals = cupy.random.multivariate_normal(self.Z_mean, self.Z_cov, (self.N_PATHS, self.N_STEPS), dtype=cupy.float32)
            ############ new>
            
            single_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, self.K, self.B, self.S0, 
                                                                                      self.sigma, self.mu, self.r, all_normals, self.N_STEPS, self.N_PATHS, self.N_STOCKS, self.s_curr)
            Y[i] = self.output.mean()

            ############ <new - combine to get X matrix
            X[i,:] = cupy.array([self.K, self.B] + self.S0.tolist() +
                                  self.sigma.tolist() + self.mu.tolist() + [self.r], dtype=cupy.float32)
            ############ new>
          
        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

ds = NumbaOptionDataSet(max_len=10, number_path=10000, batch=3, seed=15)
for i in ds:
  print(i[0])

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


tensor([[1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000e-01,
         0.0000e+00, 0.0000e+00, 5.0000e-02],
        [1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000e-01,
         0.0000e+00, 0.0000e+00, 5.0000e-02],
        [1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000e-01,
         0.0000e+00, 0.0000e+00, 5.0000e-02]], device='cuda:0')
tensor([[1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000e-01,
         0.0000e+00, 0.0000e+00, 5.0000e-02],
        [1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000e-01,
         0.0000e+00, 0.0000e+00, 5.0000e-02],
        [1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000e-01,
         0.0000e+00, 0.0000e+00, 5.0000e-02]], device='cuda:0')
tensor([[1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000e-01,
         0.0000e+00, 0.0000e+00, 5.0000e-02],
        [1.0000e+02, 9.5000e+01, 1.0000e+02, 1.0000e+02, 2.0000e-01, 2.0000

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


In [ ]:
#%%writefile cupy_dataset.py
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

@cuda.jit
def single_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_STOCKS, s_curr):

    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)    

    for i in range(ii, N_PATHS, stride): # for each path          
        running_average = 0.0

        for j in range(N_STOCKS): # initialize S0
            s_curr[j] = S0[j]

        for n in range(N_STEPS): # for each step
            s_curr_avg = 0.0

            for j in range(N_STOCKS): # for each stock
                tmp1 = mu[j]*T/N_STEPS  
                s_curr[j] += tmp1 * s_curr[j] + sigma[j]*s_curr[j]*tmp3*d_normals[i,n,j]
                s_curr_avg = s_curr_avg + 1.0/(j + 1.0) * (s_curr[j] - s_curr_avg) # S average in this step

            # add stock average to running average
            running_average = running_average + 1.0/(n + 1.0) * (s_curr_avg - running_average)

            # compare to barrier
            if running_average <= B:
                break

        payoff = running_average - K if running_average > K else 0
        #d_s[i] = tmp2 * payoff
        d_s[i] = running_average
    

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, number_stocks = 3, batch=1, threads=512, seed=15, T=1):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_STOCKS = number_stocks
        self.N_BATCH = batch
        self.T = np.float32(T)
        self.output = cupy.zeros(self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_STOCKS - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)

        ############ <new
        self.Z_mean = cupy.zeros(self.N_STOCKS, dtype=cupy.float32)
        #self.Z_cov = (-0.2 + cupy.random.rand(self.N_STOCKS*self.N_STOCKS, dtype=cupy.float32)*0.4).reshape(self.N_STOCKS,self.N_STOCKS)
        #cupy.fill_diagonal(self.Z_cov, 1)
        self.Z_cov = cupy.ones([self.N_STOCKS,self.N_STOCKS], dtype=cupy.float32)
        ############ new>

    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration

        X = cupy.zeros((self.N_BATCH, 3 + self.N_STOCKS * 3), dtype=cupy.float32)
        Y = cupy.zeros(self.N_BATCH, dtype=cupy.float32)

        for i in range(self.N_BATCH): # for each batch
          self.S0 = cupy.array([100]*self.N_STOCKS, dtype=cupy.float32)
          self.K = 100.0
          self.B = 95.0
          self.sigma = cupy.array([0.2]*self.N_STOCKS, dtype=cupy.float32)
          self.mu = cupy.array([0]*self.N_STOCKS, dtype=cupy.float32)
          self.r = 0.05
          self.s_curr = cupy.zeros(self.N_STOCKS, dtype=cupy.float32) # used to store s_curr in kernel

          ############ <new - add correlation between stocks
          all_normals = cupy.random.multivariate_normal(self.Z_mean, self.Z_cov, (self.N_PATHS, self.N_STEPS), dtype=cupy.float32)
          ############ new>
          
          single_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, self.K, self.B, self.S0, 
                                                                                    self.sigma, self.mu, self.r, all_normals, self.N_STEPS, self.N_PATHS, self.N_STOCKS, self.s_curr)
         # Y[i] = self.output.mean()
          Y[i] = self.output

          ############ <new - combine to get X matrix
          X[i,:] = cupy.array([self.K, self.B] + self.S0.tolist() +
                                self.sigma.tolist() + self.mu.tolist() + [self.r], dtype=cupy.float32)
          ############ new>
        
        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

ds = NumbaOptionDataSet(max_len=10, number_path=100, batch=1, seed=15)
for i in ds:
  print(i[1])

In [ ]:
N_STOCKS = 2
N_PATHS = 100
N_STEPS = 365

Z_mean = cupy.zeros(N_STOCKS, dtype=cupy.float32)
# Z_cov = (-0.2 + cupy.random.rand(N_STOCKS*N_STOCKS, dtype=cupy.float32)*0.4).reshape(N_STOCKS,N_STOCKS)
# cupy.fill_diagonal(Z_cov, 1)
#Z_cov = cupy.ones([N_STOCKS,N_STOCKS], dtype=cupy.float32)
Z_cov = cupy.array([[1,0.99999],[0.99999,1]], dtype=cupy.float32)

test = cupy.random.multivariate_normal(Z_mean, Z_cov, (N_PATHS, N_STEPS), dtype=cupy.float32)
np.cov(test[0,:,0], test[0,:,1])

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


array([[1.01319218, 1.0129091 ],
       [1.0129091 , 1.01264464]])

In [ ]:
mean = (1, 2)
cov = [[1, 0], [0, 1]]
x = np.random.multivariate_normal(mean, cov, (3, 3))
x

array([[[ 2.82741622,  2.44091164],
        [ 1.21183596,  2.93504818],
        [ 1.32137184,  1.39742252]],

       [[ 2.63743642,  1.22083863],
        [ 2.6331421 ,  1.35025378],
        [ 0.10744089,  3.06378457]],

       [[ 2.3769185 ,  2.06274757],
        [ 1.07737134,  2.89589748],
        [-0.14065766, -0.25167839]]])

In [ ]:
cupy.zeros(N_STOCKS, dtype=cupy.float32)

array([0., 0.], dtype=float32)

In [ ]:
cov[:,:,0]

array([[-0.0983905 , -0.32467306, -0.00850703, ..., -0.69119817,
         1.1081166 ,  1.4816111 ],
       [ 0.13093112,  0.49157935,  1.2215294 , ..., -1.7402995 ,
        -1.4249837 ,  0.14142117],
       [ 0.5419938 , -2.4765553 ,  0.37870562, ..., -0.6660495 ,
         0.49496147,  1.0302433 ],
       ...,
       [ 0.5332466 ,  0.9650368 ,  0.04600096, ..., -1.4403468 ,
        -0.34186473,  0.24416114],
       [ 1.3939893 ,  0.4824665 ,  1.0664046 , ..., -1.3792837 ,
        -0.04063462,  0.80182713],
       [ 1.0238372 , -0.91995674,  0.6804266 , ...,  0.32282877,
         0.27119708,  0.6155007 ]], dtype=float32)

In [ ]:
cupy.ones([10,5])

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [ ]:
@cuda.jit
def batch_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_STOCKS, s_curr):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp3 = math.sqrt(T/N_STEPS)
    tmp2 = math.exp(-r*T)
    for i in range(ii, N_PATHS, stride):              
        running_average = 0.0
        for j in range(N_STOCKS): # initialize S0
            s_curr[j] = S0[j]
        for n in range(N_STEPS):
            s_curr_avg = 0.0
            for j in range(N_STOCKS):
                tmp1 = mu[j]*T/N_STEPS  
                s_curr[j] += tmp1 * s_curr[j] + sigma[j]*s_curr[j]*tmp3*d_normals[i,n,j]
                s_curr_avg = s_curr_avg + 1.0/(j + 1.0) * (s_curr[j] - s_curr_avg) # S average in this step
            running_average = running_average + 1.0/(n + 1.0) * (s_curr_avg - running_average)
            if running_average <= B:
                break
        payoff = running_average - K if running_average > K else 0
        d_s[i] = tmp2 * payoff

In [ ]:
max_len=10
number_path = 1000
number_stocks = 5
batch=1
threads=512
seed=15
T=1

num = 0
max_length = max_len
N_PATHS = number_path
N_STEPS = 365
N_STOCKS = number_stocks
N_BATCH = batch
T = np.float32(T)
output = cupy.zeros(N_PATHS, dtype=cupy.float32) 
number_of_blocks = (N_PATHS * N_STOCKS - 1) // threads + 1
number_of_threads = threads
cupy.random.seed(seed)

Z_mean = cupy.zeros(N_STOCKS, dtype=cupy.float32)
Z_cov = (-0.3 + cupy.random.rand(N_STOCKS*N_STOCKS, dtype=cupy.float32)*0.6).reshape(N_STOCKS,N_STOCKS)
cupy.fill_diagonal(Z_cov, 1)

S0 = cupy.random.rand(N_STOCKS, dtype=cupy.float32) * 200
K = 110.0
B = 100.0
sigma = cupy.random.rand(N_STOCKS, dtype=cupy.float32) * 0.4
mu = cupy.random.rand(N_STOCKS, dtype=cupy.float32) * 0.2
r = 0.05
s_curr = cupy.zeros(N_STOCKS, dtype=cupy.float32) # used to store s_curr in kernel

all_normals = cupy.random.multivariate_normal(Z_mean, Z_cov, (N_PATHS, N_STEPS), dtype=cupy.float32)

In [ ]:
batch_barrier_option[(number_of_blocks,), (number_of_threads,)](output, T, K, B, S0, sigma, mu, r, all_normals, N_STEPS, N_PATHS, N_STOCKS, s_curr)
output

### Model
To map the option parameters to price, we use 6 layers of fully connected neural network with hidden dimension 512 as inspired by [this paper](https://arxiv.org/abs/1809.02233). Writing this DL price model into a file `model.py`:-

In [ ]:
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F
import torch


class Net(nn.Module):

    def __init__(self, hidden=1024):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, 1)
        self.register_buffer('norm',
                             torch.tensor([110.0, #K
                                           100.0, #B
                                           200.0, 200.0, 200.0, #S0
                                           0.2, 0.2, 0.2, #sigma
                                           0.2, 0.2, 0.2, #mu
                                           0.05])) #r

    def forward(self, x):
        # normalize the parameter to range [0-1] 
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        return self.fc6(x)

Writing model.py


As we know the random parameters' scaling factors, the input parameters are first scaled back to a range of (0-1) by dividing them by (200.0, 198.0, 200.0, 0.4, 0.2, 0.2). Then they are projected 5 times to the hidden dimension of 512 after the `ELu` activation function. `ELu` is chosen because we need to compute the second order differentiation of the parameters. If use ReLu, the second order differentiation will always be zero. The last layer is a linear layer that maps the hidden dimension to the predicted option price. 

For training, we use [Ignite](https://github.com/pytorch/ignite) which is a high-level library to train neural networks in PyTorch. We use `MSELoss` as the loss function, `Adam` as the optimizer and `CosineAnnealingScheduler` as the learning rate scheduler. The following code is feeding the random option data to the pricing model to train it.

In [ ]:
!pip install pytorch-ignite

     |████████████████████████████████| 204kB 4.2MB/s 


In [ ]:
from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import NumbaOptionDataSet
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
dataset = NumbaOptionDataSet(max_len=100, number_path = 1024, batch=2)
#dataset = OptionDataSet(max_len=10000, number_path = 1024, batch=4800)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value())
        
trainer.run(dataset, max_epochs=100)

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


loss 114.11465454101562 average time 0.004820869999996376
loss 103.97425842285156 average time 0.00477378225999928
loss 65.08654022216797 average time 0.004628413440002532
loss 1.4812583923339844 average time 0.004637326459992437
loss 0.5752648115158081 average time 0.0046728680700061885


Engine run is terminating due to exception: 


KeyboardInterrupt: ignored

The loss is keeping decreasing which means the pricing model can predict the option prices better. It takes about $12ms$ to compute one mini-batch in average, In the following sections, we will try to expore the full potentials of the GPU to accelerate the training.

### TensorCore mixed precision training

The V100 GPUs have 640 tensor cores that can accelerate half precision matrix multiplication calculation which is the core computation done by the DL model. [Apex library](https://github.com/NVIDIA/apex) developed by NVIDIA makes mixed precision and distributed training in Pytorch easy. By changing 3 lines of code, it can use the tensor cores to accelerate the training. 

In [ ]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 8042, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 8042 (delta 61), reused 69 (delta 30), pack-reused 7913
Receiving objects: 100% (8042/8042), 14.11 MiB | 11.67 MiB/s, done.
Resolving deltas: 100% (5460/5460), done.


In [ ]:
cd apex

/content/apex


In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-ai1_gori
Created temporary directory: /tmp/pip-req-tracker-vtnptd92
Created requirements tracker '/tmp/pip-req-tracker-vtnptd92'
Created temporary directory: /tmp/pip-install-657rtgx2
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-v0dzb7mt
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-vtnptd92'
    Running setup.py (path:/tmp/pip-req-build-v0dzb7mt/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-v0dzb7mt/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-v0dzb7mt/pip-egg-info/apex.egg-info/PK

In [ ]:
from apex import amp
from ignite.engine import Engine, Events
from torch.nn import MSELoss
from ignite.handlers import Timer
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import NumbaOptionDataSet
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
# set the AMP optimization level to O1
opt_level = 'O1'
# wrap the optimizer and model
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
dataset = NumbaOptionDataSet(max_len=100, number_path = 1024, batch=2)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    # amp handles the auto loss scaling
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value())
        
trainer.run(dataset, max_epochs=100)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0
loss 198.99514770507812 average time 0.007484014839999418
loss 77.05516815185547 average time 0.007719316529999105
loss 46.65948486328125 average time 0.007572146340005474
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 128.0
loss 0.5194532871246338 average time 0.007530288619989278
loss 1.6039482355117798 average time 0.007623035989990967
lo

Engine run is terminating due to exception: 


KeyboardInterrupt: ignored

It improves to compute each mini-batch in $8ms$. As we reduce the model weights to half precision for better performance, the loss need to be scaled to make sure the half precision dynamic range aligns with the computation. It is guessing what is the correct loss scaling factor and adjust it automatically if the gradient overflows. In the end, we will get the best hardware acceleration while maintaining the accuracy of model prediction.

### Multiple GPU training

Apex makes multiple GPU training easy. Working on the same training script, we need to take care of a few extra steps:

1. Add the argument `--local_rank` which will be automatically set by the distributed launcher
2. Initialize the process group
2. Generate independent batched data based on process id in the dataset.
3. Wrap the model and optimizer to handle distributed computation. 
4. Scale the loss and optimizer

To launch distributed training, we need to put everything into a python file. Following is an example:-

In [ ]:
pwd

'/content/apex'

In [ ]:
cd ..

/


In [ ]:
%%writefile distributed_train.py 
import cupy
import numpy as np
import math
import time
import os
import torch
from torch.utils.dlpack import from_dlpack
import torch.nn as nn
import torch.nn.functional as F
import torch
from apex import amp
from ignite.engine import Engine, Events
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from apex.parallel import DistributedDataParallel 
import argparse
from model import Net
from cupy_dataset import OptionDataSet

parser = argparse.ArgumentParser()
parser = argparse.ArgumentParser()
# this local_rank arg is automaticall set by distributed launch
parser.add_argument("--local_rank", default=0, type=int)
args = parser.parse_args()

args.distributed = False
if 'WORLD_SIZE' in os.environ:
    args.distributed = int(os.environ['WORLD_SIZE']) > 1

if args.distributed:
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl',
                                         init_method='env://')

torch.backends.cudnn.benchmark = True


model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
if args.distributed:
    model = DistributedDataParallel(model)
dataset = OptionDataSet(max_len=10000, number_path = 1024, batch=10240, seed=args.local_rank)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output)
        
trainer.run(dataset, max_epochs=100)

Overwriting distributed_train.py


To launch multiple processes training, we need to run the following command:-

In [ ]:
%reset -f

!python -m torch.distributed.launch --nproc_per_node=4 distributed_train.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Killing subprocess 290
Killing subprocess 291
Killing subprocess 292
Killing subpr

It works and all the GPUs are busy to train this network. However, it has a few problems:-
   
    1. There is no model serialization so the trained model is not saved
    2. There is no validation dataset to check the training progress
    3. Most of the time is spent in Monte Carlo simulation hence the training is slow
    4. We use a few paths(1024) for each option parameter set which is noise and the model cannot converge to a low cost value.
We will address these problems in the next notebook